In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from datetime import datetime
import xarray as xr
import requests
from bs4 import BeautifulSoup
import copy

## Data Preparation

**Purpose of this notebook:** In this notebook, we pull the data required for this project. The output from the below code will be the file `dat.csv`.

Structure of this notebook:
1. Severe Weather Event Data: In this section we download severe weather event data in the United States from the following website: https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/ 
2. Weather Data: In this section we download weather data in the United States from the following website: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=overview
3. Merging Severe Weather Event Data and Weather Data: At the end, we will merge the two data sets by latitude and longitude.

## 1. Severe Weather Event Data

### 1.1. Data Download (Web Cralling)

In [ ]:
# lint to csv files
link = "https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/"
response = requests.get(link)
html = response.text

# structure the website with BeautifulSoup
soup = BeautifulSoup(html)

# create list of file names ("a" for links)
elements = soup.findAll("a")

# download
all_dat = []

# last 2 elements not relevant; first x elements for data starting 2010
for element in elements[68:-2]:
    # parse element for ending part of url
    if element.attrs["href"].startswith("StormEvents_details"):
        # create url
        filename = element.attrs["href"]
        full_url = link + filename
        
        # load the file
        csv = pd.read_csv(full_url, compression = "gzip")
        
        # append to to storm_event_details_allyears
        all_dat.append(csv)

# concat the pulled data
severe_events = pd.concat(all_dat)

# create csv file
severe_events.to_csv("severe_events.csv")

### 1.2. Data Preparation

In [5]:
# load severe data
severe_events = pd.read_csv("datasets/severe_events.csv", index_col = "Unnamed: 0")

# print info on dataframe
print("The dataset has {} rows and {} columns.".format(severe_events.shape[0], severe_events.shape[1]))
print(severe_events.head())

The dataset has 630629 rows and 51 columns.
   BEGIN_YEARMONTH  BEGIN_DAY  BEGIN_TIME  END_YEARMONTH  END_DAY  END_TIME  \
0           201007          7        1251         201007        7      1630   
1           201001         17        2300         201001       18      1500   
2           201010          1         830         201010        1      1000   
3           201007          6         951         201007        6      1830   
4           201012         26        1700         201012       27      1800   

   EPISODE_ID  EVENT_ID          STATE  STATE_FIPS  ...  END_RANGE  \
0       43850    254780  NEW HAMPSHIRE          33  ...        NaN   
1       36500    211550  NEW HAMPSHIRE          33  ...        NaN   
2       44854    260014  NEW HAMPSHIRE          33  ...        NaN   
3       43850    254779  NEW HAMPSHIRE          33  ...        NaN   
4       46989    273769  NEW HAMPSHIRE          33  ...        NaN   

  END_AZIMUTH END_LOCATION BEGIN_LAT  BEGIN_LON END_LAT END_

In [6]:
# clean date
def clean_date_1(dat):
    # convert to string
    dat["BEGIN_DAY"] = dat["BEGIN_DAY"].astype(str)
    dat["BEGIN_YEARMONTH"] = dat["BEGIN_YEARMONTH"].astype(str)

    # make day two digits
    dat["BEGIN_DAY"] = [str("0"+i) if len(i) == 1 else i for i in dat["BEGIN_DAY"]]

    # concat strings together
    dat["date"] = dat["BEGIN_YEARMONTH"].astype(str) + dat["BEGIN_DAY"].astype(str)

    # convert date
    date_time_objects = [datetime.strptime(i, '%Y%m%d') for i in dat["date"]]
    dat["date"] = [i.strftime("%m/%d/%Y") for i in date_time_objects]
    
    # convert to date time
    dat["date"] = pd.to_datetime(dat["date"], format="%m/%d/%Y", errors='ignore')
    
    # drop other time columns
    dat.drop(columns=["BEGIN_YEARMONTH", "BEGIN_DAY", "BEGIN_TIME", "END_YEARMONTH", "END_DAY", "END_TIME"], inplace = True)
    
    # return df
    return dat

In [7]:
# drop irrelevant columns
def drop_cols(dat):
    # drop
    dat.drop(columns=["EVENT_ID", "EPISODE_ID", "STATE_FIPS", "YEAR", "MONTH_NAME", "CZ_TYPE", "CZ_FIPS", "CZ_NAME", "WFO", 
                      "BEGIN_DATE_TIME", "CZ_TIMEZONE", "END_DATE_TIME", "SOURCE", "TOR_OTHER_WFO", "TOR_OTHER_CZ_STATE", 
                      "TOR_OTHER_CZ_FIPS", "TOR_OTHER_CZ_NAME", "BEGIN_RANGE", "BEGIN_AZIMUTH", "BEGIN_LOCATION", 
                      "END_RANGE", "END_AZIMUTH", "END_LOCATION", "DATA_SOURCE", "EPISODE_NARRATIVE", "EVENT_NARRATIVE", 
                      "FLOOD_CAUSE", "CATEGORY", "MAGNITUDE", "MAGNITUDE_TYPE", "END_LAT", "END_LON"], inplace = True)
    
    # return df
    return dat

In [8]:
# select category
def sel_cat(dat, category):
    # select category
    if len(category) == 0:
        pass
    else:
        # select
        dat = dat.loc[dat["EVENT_TYPE"].isin(category), ].copy()
        # reset index
        dat.reset_index(inplace = True, drop = True)
    
    # return
    return dat

In [9]:
# select timeframe
def timeframe(dat, start_date, end_date):
    # select date
    dat = dat[(dat["date"] > start_date) & (dat["date"] < end_date)]
    
    # reset index
    dat.reset_index(inplace = True, drop = True)
    
    # return
    return dat

In [10]:
# convert damage columns into integers
def damage_int(dat):
    # initialize dict
    dct = dct = {"B": "0000000", "M": "0000", "K": "0", "0": "0"}
    
    # cols to convert
    cols = ["DAMAGE_PROPERTY", "DAMAGE_CROPS"]
    
    # iterate over cols
    for col in cols:
        scale = [str(i)[-1] if type(i) == str else i for i in dat[col]]
        dec_2 = [str(i)[-2] if type(i) == str else i for i in dat[col]]
        dec_1 = [str(i)[-3] if type(i) == str else i for i in dat[col]]
        integer = [str(i)[:-4] if type(i) == str else i for i in dat[col]]
        temp = [a + b + c + dct[d] if (type(a) == str) & (type(b) == str) & (type(c) == str) & (type(d) == str) else np.nan for a,b,c,d in zip(integer,dec_1,dec_2,scale)]
        dat[col] = [float(i) for i in temp]
        
    # return
    return dat

In [11]:
# remove nans for lat/long (nans occur by category; all or nothing)
def remove_nan(dat):
    # drop
    dat = dat.dropna(subset = ["BEGIN_LAT", "BEGIN_LON"])
    
    # reset index
    dat.reset_index(drop = True, inplace = True)
    
    # return
    return dat

In [12]:
# run functions
severe_events = clean_date_1(severe_events)
severe_events = drop_cols(severe_events)
severe_events = timeframe(severe_events, "2015-01-01", "2020-01-01")
severe_events = sel_cat(severe_events, [])
severe_events = damage_int(severe_events)
severe_events = remove_nan(severe_events)

### 1.3. Data Exploration

In [13]:
# number of data points by category
print(severe_events["EVENT_TYPE"].value_counts())

Thunderstorm Wind           79766
Hail                        46079
Flash Flood                 20761
Flood                       16535
Marine Thunderstorm Wind    10733
Heavy Rain                   8124
Tornado                      7023
Lightning                    1916
Funnel Cloud                 1603
Waterspout                   1057
Debris Flow                   617
Marine High Wind              237
Marine Hail                   138
Marine Strong Wind             54
Dust Devil                     45
Name: EVENT_TYPE, dtype: int64


In [14]:
# damage by event
severe_events.groupby(["EVENT_TYPE"]).sum()

,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,TOR_LENGTH,TOR_WIDTH,BEGIN_LAT,BEGIN_LON
EVENT_TYPE,,,,,,,,,,
Debris Flow,171,1,27,3,6.793275e+08,9.100000e+03,0.00,0.00,2.323239e+04,-7.085176e+04
Dust Devil,10,20,0,0,7.397000e+05,0.000000e+00,0.00,0.00,1.706579e+03,-4.911185e+03
Flash Flood,155,19,447,27,7.072191e+10,5.487728e+08,0.00,0.00,7.533910e+05,-1.900097e+06
Flood,43,8,175,14,1.077876e+10,2.032296e+09,0.00,0.00,6.432662e+05,-1.498740e+06
Funnel Cloud,2,0,0,0,1.000000e+03,0.000000e+00,0.00,0.00,6.291997e+04,-1.555916e+05
Hail,47,27,0,3,7.621878e+09,3.229358e+08,0.00,0.00,1.790255e+06,-4.380627e+06
Heavy Rain,11,92,11,34,1.380032e+08,1.533661e+07,0.00,0.00,3.011886e+05,-7.983079e+05
Lightning,518,71,128,9,1.126386e+08,2.244000e+04,0.00,0.00,6.857083e+04,-1.680327e+05
Marine Hail,0,0,0,0,0.000000e+00,0.000000e+00,0.00,0.00,5.429001e+03,-1.141310e+04


The categories Thuderstorm Wind, Hail, Flash Flood, Marine Thunderstorm Wind, Heavy Wind, Tornado, and Lightning will be used for the subsequent analysis given their number of data points combined with their severity in terms of injuries, deaths, and damage.

In [15]:
# select relevant categories
severe_events = sel_cat(severe_events, ["Thunderstorm Wind", "Hail", "Flash Flood", "Flood", "Marine Thunderstorm Wind", "Heavy Rain", "Tornado", "Lightning"])

In [16]:
# display df
print("The final dataset has {} rows and {} columns.".format(severe_events.shape[0], severe_events.shape[1]))
severe_events.head()

The final dataset has 190937 rows and 14 columns.


,STATE,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,BEGIN_LAT,BEGIN_LON,date
0,GEORGIA,Thunderstorm Wind,0,0,0,0,2000.0,0.0,NaN,NaN,NaN,32.94,-82.20,2015-09-05
1,SOUTH CAROLINA,Thunderstorm Wind,0,0,0,0,14000.0,0.0,NaN,NaN,NaN,34.71,-80.72,2015-09-10
2,GEORGIA,Thunderstorm Wind,0,0,0,0,2000.0,0.0,NaN,NaN,NaN,32.94,-82.14,2015-09-04
3,GEORGIA,Thunderstorm Wind,0,0,0,0,3000.0,0.0,NaN,NaN,NaN,33.21,-82.15,2015-09-05
4,SOUTH CAROLINA,Thunderstorm Wind,0,0,0,0,12000.0,0.0,NaN,NaN,NaN,34.72,-80.75,2015-09-10


## 2. Weather Data

### 2.1. Data Download (API Call)

In [17]:
# estimation of the size of the data
years = 10
print("{} years of weather data of the entire US has a size of approx. {:.2f}GB and {:.2f} million rows of data.".format(years, 0.742*365*years/1000, 22968*365*years/1000000))

10 years of weather data of the entire US has a size of approx. 2.71GB and 83.83 million rows of data.


In [ ]:
# download weather data from 2010-2019
# requirements: make account with copernicus to get user id; follow instrucitons: https://cds.climate.copernicus.eu/api-how-to

import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'variable': [
            'divergence', 'fraction_of_cloud_cover', 'geopotential',
            'ozone_mass_mixing_ratio', 'potential_vorticity', 'relative_humidity',
            'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 'specific_humidity',
            'specific_rain_water_content', 'specific_snow_water_content', 'temperature',
            'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity',
            'vorticity',
        ],
        'pressure_level': '500',
        'year': [
            '2010', '2011', '2012',
            '2013', '2014', '2015',
            '2016', '2017', '2018',
            '2019',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': '12:00',
        'format': 'netcdf',
        'area': [
            49.35, -124.78, 24.74,
            -66.95,
        ],
    },
    'dat.nc')

### 2.2. Data Manipulation
- This step was performed on Google Colab due to RAM issues of our PCs
- See google colab notebook https://colab.research.google.com/drive/1flZy0Y8h-zg8qXVUnpGBdQ6tU-GBDeGO?authuser=1#scrollTo=uXnVHL657gnh
- Manupulations performed: Decreased the number of center points by enlarging the distance between latitude and longitude checkpoints; select data form 2015-2020 only

### 2.3. Data Prepartion

In [18]:
# load data
weather = pd.read_csv("datasets/weather.csv", index_col = "Unnamed: 0")

# print info on dataframe
print("The dataset has {} rows and {} columns.".format(weather.shape[0], weather.shape[1]))
print(weather.head())

/Users/juliankleindiek/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


The dataset has 4091652 rows and 19 columns.
   latitude   longitude                 time         d        cc          z  \
0      49.1 -124.779999  2015-01-02 12:00:00 -0.000016  0.070316  54890.785   
1      49.1 -124.779999  2015-01-03 12:00:00 -0.000014  0.000000  54526.414   
2      49.1 -124.779999  2015-01-04 12:00:00  0.000007  0.687501  54611.207   
3      49.1 -124.779999  2015-01-05 12:00:00 -0.000019  0.828132  55472.645   
4      49.1 -124.779999  2015-01-06 12:00:00 -0.000067  0.000000  56132.560   

             o3            pv          r      ciwc          clwc         q  \
0  5.880813e-08  2.492025e-07  83.549900  0.000009  0.000000e+00  0.001121   
1  6.822363e-08  2.810552e-07  91.006680  0.000000  0.000000e+00  0.001085   
2  7.065951e-08  2.848474e-07  95.850685  0.000040  0.000000e+00  0.001296   
3  6.970812e-08  2.909146e-07  94.503580  0.000068  2.706112e-05  0.002280   
4  7.862332e-08  6.860441e-07  84.408420  0.000000  2.358574e-07  0.001688   

   crwc    

In [19]:
# clean date
def clean_date_2(dat):
    # change format
    date_time_objects = [datetime.strptime(str(i), '%Y-%m-%d %H:%M:%S') for i in dat["time"]]
    dat["date"] = [i.strftime("%m/%d/%Y") for i in date_time_objects]
    
    # create
    dat["date"] = pd.to_datetime(dat["date"], format="%m/%d/%Y", errors='ignore')
    
    # drop other time columns
    dat.drop(columns=["time"], inplace = True)
    
    # return
    return dat

In [20]:
# run functions
weather = clean_date_2(weather)

In [21]:
# display df
print("The final dataset has {} rows and {} columns.".format(weather.shape[0], weather.shape[1]))
weather.head()

The final dataset has 4091652 rows and 19 columns.


,latitude,longitude,d,cc,z,o3,pv,r,ciwc,clwc,q,crwc,cswc,t,u,v,w,vo,date
0,49.1,-124.779999,-0.000016,0.070316,54890.785,5.880813e-08,2.492025e-07,83.549900,0.000009,0.000000e+00,0.001121,0.0,0.000024,253.58159,27.670355,-4.670503,-0.327145,-0.000049,2015-01-02
1,49.1,-124.779999,-0.000014,0.000000,54526.414,6.822363e-08,2.810552e-07,91.006680,0.000000,0.000000e+00,0.001085,0.0,0.000000,252.46490,27.884556,-16.975382,0.001749,-0.000040,2015-01-03
2,49.1,-124.779999,0.000007,0.687501,54611.207,7.065951e-08,2.848474e-07,95.850685,0.000040,0.000000e+00,0.001296,0.0,0.000017,253.62744,28.948296,-5.151333,-0.291906,-0.000026,2015-01-04
3,49.1,-124.779999,-0.000019,0.828132,55472.645,6.970812e-08,2.909146e-07,94.503580,0.000068,2.706112e-05,0.002280,0.0,0.000113,259.64148,37.004395,-2.560414,-0.617444,-0.000069,2015-01-05
4,49.1,-124.779999,-0.000067,0.000000,56132.560,7.862332e-08,6.860441e-07,84.408420,0.000000,2.358574e-07,0.001688,0.0,0.000007,257.61942,25.296001,4.286448,-0.942983,-0.000044,2015-01-06


### 2.4. Data Exploration

In [22]:
# describe
weather.describe()

,latitude,longitude,d,cc,z,o3,pv,r,ciwc,clwc,q,crwc,cswc,t,u,v,w,vo
count,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06,4.091652e+06
mean,3.681360e+01,-9.599708e+01,1.059006e-08,6.380568e-02,5.620193e+04,9.695186e-08,4.517613e-07,4.143982e+01,4.062693e-06,3.845405e-06,1.047064e-03,1.784447e-09,1.337259e-05,2.593736e+02,1.185923e+01,-2.003868e-01,2.038312e-03,-1.534947e-06
std,7.155192e+00,1.673024e+01,2.629769e-05,1.899079e-01,1.603139e+03,2.164509e-08,3.749405e-07,3.064085e+01,1.518939e-05,1.938521e-05,9.615275e-04,3.723471e-07,7.366875e-05,7.104009e+00,1.104522e+01,1.003748e+01,3.222157e-01,5.137911e-05
min,2.474000e+01,-1.247800e+02,-5.202713e-04,0.000000e+00,4.720367e+04,-3.729497e-09,-3.702093e-06,-5.368446e+00,0.000000e+00,0.000000e+00,1.583248e-08,0.000000e+00,0.000000e+00,2.199214e+02,-4.199187e+01,-6.057594e+01,-1.342417e+01,-5.859006e-04
25%,3.062000e+01,-1.105000e+02,-1.258055e-05,0.000000e+00,5.534545e+04,8.183017e-08,2.207621e-07,1.439610e+01,0.000000e+00,0.000000e+00,3.208816e-04,0.000000e+00,0.000000e+00,2.555821e+02,3.919558e+00,-5.612294e+00,-9.557676e-02,-3.195577e-05
50%,3.650000e+01,-9.538000e+01,3.812165e-07,0.000000e+00,5.662607e+04,9.226267e-08,3.792688e-07,3.376512e+01,0.000000e+00,0.000000e+00,7.230579e-04,0.000000e+00,0.000000e+00,2.609227e+02,1.097364e+01,2.454519e-02,2.691937e-02,-9.672425e-06
75%,4.322000e+01,-8.110000e+01,1.312299e-05,7.812858e-03,5.743997e+04,1.081657e-07,5.863135e-07,6.545819e+01,2.383022e-07,0.000000e+00,1.483796e-03,0.000000e+00,1.048669e-06,2.648643e+02,1.882099e+01,5.434877e+00,1.418643e-01,1.820439e-05
max,4.910000e+01,-6.766000e+01,5.811859e-04,1.000000e+00,5.881993e+04,4.576546e-07,1.422283e-05,1.414825e+02,6.318094e-04,9.279242e-04,8.257235e-03,2.501759e-04,5.726933e-03,2.782005e+02,7.395573e+01,6.281460e+01,7.150160e+00,1.275574e-03


## 3. Weather Data and Severe Weather Event Data

### 3.1. Merge Datasets

In [23]:
# grid match of event to weather
def grid_match(event, weather):
    # latitude
    aa = event["BEGIN_LAT"]
    bb = weather["latitude"].unique()
    event["latitude_new"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]
    
    # longitude
    aa = event["BEGIN_LON"]
    bb = weather["longitude"].unique()
    event["longitude_new"] = bb[abs(aa[None, :] - bb[:, None]).argmin(axis=0)]
    
    # return
    return event

In [24]:
# merge dataframes
def merge_dfs(event, weather):
    # merge
    df_merged = weather.merge(event, left_on=['latitude','longitude', 'date'], right_on=['latitude_new','longitude_new', 'date'], how='left')
    
    # drop duplicate lat/longs
    df_merged.drop(columns=['latitude_new','longitude_new'], inplace = True)
    
    # return
    return df_merged

In [37]:
# add column with information on whether row includes target values or not
def target(dat):
    dat["target"] = [0 if np.isnan(i) else 1 for i in dat["BEGIN_LAT"]]
    return dat

In [38]:
# run functions
severe_events = grid_match(severe_events, weather)
dat = merge_dfs(severe_events, weather)
dat = target(dat)

In [39]:
# display data
dat.head()

,latitude,longitude,d,cc,z,o3,pv,r,ciwc,clwc,...,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,BEGIN_LAT,BEGIN_LON,target
0,49.1,-124.779999,-0.000016,0.070316,54890.785,5.880813e-08,2.492025e-07,83.549900,0.000009,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,49.1,-124.779999,-0.000014,0.000000,54526.414,6.822363e-08,2.810552e-07,91.006680,0.000000,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,49.1,-124.779999,0.000007,0.687501,54611.207,7.065951e-08,2.848474e-07,95.850685,0.000040,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,49.1,-124.779999,-0.000019,0.828132,55472.645,6.970812e-08,2.909146e-07,94.503580,0.000068,2.706112e-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,49.1,-124.779999,-0.000067,0.000000,56132.560,7.862332e-08,6.860441e-07,84.408420,0.000000,2.358574e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


### 3.2. Remove Duplicate Data Points for the Same Events

In [40]:
# create ids for the same events
def idx_counter(dat):

    # sort dat
    dat = dat.sort_values(by=['date', 'latitude', "longitude"])

    # initialize values
    counter = 0
    idx_lst = []

    # slice data
    temp = dat[["date", "latitude", "longitude"]].values

    # create index for events with same date, lat, and long
    for idx, i in enumerate(temp):
        if idx == len(temp)-1:
            idx_lst.append(counter)
            break
        elif (i[0] == temp[idx+1][0]) & (i[1] == temp[idx+1][1]) & (i[2] == temp[idx+1][2]):
            idx_lst.append(counter)
        else:
            idx_lst.append(counter)
            counter += 1

    # create id column in dat
    dat["event_id"] = idx_lst
    
    # return
    return dat

In [68]:
# group same events, aggregate their characteristics, drop duplicate event rows
def drop_dupes_on_event_id(dat):
    """
    Note: This function may lead to a change of the calssification of an event or state.
    Assume we have a single event with multiple rows that have the same date, latitude, and longitude.
    Further assume that these rows classified the events differently, one is Thurderstorm, the other is Tornado. 
    Further assume that the event happened at the baorder of two states, one is in Georgia, the other is in Florida. 
    Now, while we can aggregate quantitative features of the multiple rows, we cannot aggregate qualitative features like type or state. 
    Finally, this leads to a situation in which we finally assign the event to the type and state that is merged first.
    This doesn't affect the total number of cases, only the labeling. This is not a porblem, because the types will be very similar and the location as well.
    """
    
    # initialize dictionary for aggregation
    agg_dct = {
        # mean 
        'd':"mean", 'cc':"mean", 'z':"mean", 'o3':"mean", 'pv':"mean", 'r':"mean", 'ciwc':"mean", 'clwc':"mean", 'q':"mean", 'crwc':"mean", 'cswc':"mean",
        't':"mean", 'u':"mean", 'v':"mean", 'w':"mean", 'vo':"mean", 'TOR_LENGTH':"mean", 'TOR_WIDTH':"mean", 'BEGIN_LAT':"mean", 'BEGIN_LON':"mean",
        # sum
        'INJURIES_DIRECT':'sum', 'INJURIES_INDIRECT':'sum', 'DEATHS_DIRECT':'sum', 'DEATHS_INDIRECT':'sum', 'DAMAGE_PROPERTY':'sum', 'DAMAGE_CROPS':'sum'}

    # select qualitative columns from original
    dat_qual_cols = copy.deepcopy(dat[['event_id', 'latitude', 'longitude', 'date', 'STATE', 'EVENT_TYPE', 'TOR_F_SCALE', 'target']])

    # group by event_id and aggregate quantitative columns using agg_dict
    dat_agg_cols = dat.groupby("event_id", as_index=False).agg(agg_dct)
    
    # merge df with qual and agg cols
    temp = dat_qual_cols.merge(dat_agg_cols, left_on='event_id', right_on='event_id')

    # did we lose columns
    if len(temp.columns) != len(dat.columns):
        print("Error! We lost columns")
        
    # drop duplicates
    dat = temp.drop_duplicates(subset = "event_id", keep="first")
    
    # reset index
    dat.reset_index(drop=True, inplace=True)
    
    # return
    return dat

In [69]:
# test function on small subset
tst = dat[(dat["EVENT_TYPE"] == "Tornado") & (dat["STATE"] == "GEORGIA")].copy()
print(tst[["date", "latitude", "longitude"]].head())
tst = idx_counter(tst)
print(tst[["date", "latitude", "longitude", "event_id"]].head())
tst = drop_dupes_on_event_id(tst)
print(tst[["date", "latitude", "longitude", "event_id"]].head())

              date  latitude  longitude
3116970 2015-01-04     31.46 -83.619999
3116971 2015-01-04     31.46 -83.619999
3119058 2015-01-04     31.46 -82.779999
2987236 2015-01-04     32.30 -83.619999
2987237 2015-01-04     32.30 -83.619999
              date  latitude  longitude  event_id
3116970 2015-01-04     31.46 -83.619999         0
3116971 2015-01-04     31.46 -83.619999         0
3119058 2015-01-04     31.46 -82.779999         1
2987236 2015-01-04     32.30 -83.619999         2
2987237 2015-01-04     32.30 -83.619999         2
        date  latitude  longitude  event_id
0 2015-01-04     31.46 -83.619999         0
1 2015-01-04     31.46 -82.779999         1
2 2015-01-04     32.30 -83.619999         2
3 2015-01-04     32.30 -81.099999         3
4 2015-04-03     34.82 -85.299999         4


In [70]:
# run functions
dat = idx_counter(dat)
dat = drop_dupes_on_event_id(dat)

In [71]:
# check if there are different states in same event_id
tst = dat.groupby("event_id").agg({"STATE": "count"})
print("There are {} event_ids with multiple states.".format(len(tst[tst["STATE"] >1])))

There are 0 event_ids with multiple states.


### 3.3. Save Final Dataset

In [72]:
# balance of dataset
print("There are {} severe weather events, representing {:.2f}% of the total dataset.".format(len(dat[dat["target"] == 1]), len(dat[dat["target"] == 1])/len(dat)*100))

There are 61719 severe weather events, representing 1.63% of the total dataset.


In [73]:
# save as csv (optional)
#dat.to_csv("datasets/dat.csv")

## 4. Misc Tests

### Variance of Features by Pressure Level
- According to https://www.climateprediction.net/climate-science/glossary/vertical-resolutions-levels/ 950 hPa is near the surface

In [26]:
# load .nc file 
tst = xr.open_mfdataset("tst.nc")
tst = tst.to_dataframe()
tst.reset_index(inplace=True)
tst.groupby(["level"]).std()

/Users/juliankleindiek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  
/Users/juliankleindiek/anaconda3/lib/python3.7/site-packages/xarray/backends/api.py:941: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


,latitude,longitude,d,cc,z,o3,pv,r,ciwc,clwc,q,crwc,cswc,t,u,v,w,vo
level,,,,,,,,,,,,,,,,,,
1,7.144501,16.743367,0.000023,0.000000,2467.747556,2.886548e-07,2.082979e-03,0.000000,0.000000e+00,0.000000e+00,6.391453e-08,0.000000e+00,0.000000e+00,5.582429,26.844506,16.437425,0.000827,0.000047
2,7.144501,16.743367,0.000026,0.000000,1489.224798,7.088156e-07,1.087185e-03,0.000465,0.000000e+00,0.000000e+00,1.262044e-07,0.000000e+00,0.000000e+00,8.424631,19.147092,10.321514,0.001408,0.000036
3,7.144501,16.743367,0.000027,0.000000,1513.928994,8.608851e-07,9.547526e-04,0.001367,0.000000e+00,0.000000e+00,1.707039e-07,0.000000e+00,0.000000e+00,6.311637,16.550284,12.530119,0.001926,0.000038
5,7.144501,16.743367,0.000025,0.000000,1680.703573,9.312160e-07,5.607121e-04,0.008675,0.000000e+00,0.000000e+00,2.716610e-07,0.000000e+00,0.000000e+00,7.023968,26.884445,11.912679,0.002805,0.000041
7,7.144501,16.743367,0.000025,0.000000,1665.048423,1.126016e-06,3.029423e-04,0.033282,0.000000e+00,0.000000e+00,2.153234e-07,0.000000e+00,0.000000e+00,5.363485,27.679144,13.536710,0.003434,0.000039
10,7.144501,16.743367,0.000022,0.000000,1518.640922,1.287521e-06,1.194316e-04,0.121976,0.000000e+00,0.000000e+00,2.320634e-07,0.000000e+00,0.000000e+00,3.689771,22.546034,10.479227,0.004475,0.000032
20,7.144501,16.743367,0.000019,0.000000,946.685633,7.082206e-07,1.224790e-05,0.647922,0.000000e+00,0.000000e+00,6.939848e-08,0.000000e+00,0.000000e+00,3.420780,9.152801,5.437986,0.007297,0.000014
30,7.144501,16.743367,0.000023,0.000000,700.372468,7.865225e-07,7.754085e-06,0.964460,0.000000e+00,0.000000e+00,1.209777e-07,0.000000e+00,0.000000e+00,3.303154,7.207565,3.725480,0.009587,0.000014
50,7.144501,16.743367,0.000027,0.000000,821.922137,1.151064e-06,6.414772e-06,4.743857,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.292856,9.054605,5.518807,0.016543,0.000022
